# Artificial Fast Fading from RIS

_Author:_ Karl-Ludwig Besser, Technische Universität Braunschweig

This notebook is part of the paper "Artificial Fast Fading from
Reconfigurable Surfaces Enables Ultra-Reliable Communications" (Eduard Jorswieck, Karl-Ludwig Besser, and Cong Sun. IEEE SPAWC 2021).

If you use any of this work, please cite the above paper.

> If you are not familiar with Jupyter notebooks: The easiest way to use this notebook interactively, is to hit `Kernel --> Restart & Run All` in the menu. This will execute all cells and enable the interactive elements of the plots.  
> Alternatively, you can execute the cells one by one using Shift+Return

In [4]:
import numpy as np
from scipy import stats
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual

In [5]:
from phases import gains_constant_phase, rvs_channel_phases, rvs_ris_phases, rvs_ris_phases_quant

# Constant RIS Phases

First, the all RIS phases are fixed to the constant value $\theta_i=0$.

In [6]:
def constant_ris_phases(num_samples=50000, num_elements=[5, 10, 20, 50]):
    fig, axs = plt.subplots()
    def update(show_approx=True):
        axs.clear()
        axs.set_xlabel("Rate $R$")
        axs.set_ylabel("Outage Probability $\\varepsilon$")
        for _num_elements in num_elements:
            channel_realizations = rvs_channel_phases(_num_elements, num_samples)
            const_phase = gains_constant_phase(channel_realizations)
            capac_const_phase = np.log2(1 + const_phase)
            _hist = np.histogram(capac_const_phase, bins=100)
            _r_ax = np.linspace(min(capac_const_phase)*.9, max(capac_const_phase)*1.1, 1000)
            cdf_hist = stats.rv_histogram(_hist).cdf(_r_ax)
            axs.plot(_r_ax, cdf_hist, label="Empirical CDF N={:d}".format(_num_elements))
            if show_approx:
                cdf_appr = 1. - np.exp(-(2**_r_ax-1)/_num_elements)  # N --> oo, for sum
                axs.plot(_r_ax, cdf_appr, '--', label="Approximate N={:d}".format(_num_elements))
        axs.legend()
    interact(update, show_approx=True)

The plot below shows the outage probability for this scenario with fixed RIS phases.

In [8]:
constant_ris_phases(num_elements=[5, 10, 20, 50])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Checkbox(value=True, description='show_approx'), Output()), _dom_classes=('widget-intera…

# Random RIS Phases

Next, we consider phase hopping at the RIS, i.e., the phases $\theta_i$ of the individual elements are varyied randomly with each transmitted symbol.

In the beginning, we consider continuous phases which are uniformly distributed over $[0, 2\pi]$.

## Two-Element RIS

First, we set the number of RIS elements to $N=2$.

In [9]:
def cdf_ergodic_capac_two_elements(rate, copula="indep"):
    if copula.startswith("comon"):
        #pdf = 2./(np.pi*np.sqrt(2-2**rate))
        cdf = 2/np.pi * np.arcsin(0.5 * np.sqrt(2**rate-1))
        cdf[np.isnan(cdf)] = 1.
    elif copula.startswith("indep"):
        #ergodic = -(np.exp(1/2)*special.expi(-1/2))/(np.log(2)) #approximation
        ergodic = np.arccosh(3/2)/np.log(2)
        cdf = np.heaviside(rate-ergodic, .5)
    elif copula.startswith("counter"):
        ergodic = np.arccosh(3/2)/np.log(2)
        cdf = np.heaviside(rate-ergodic, .5)
    return cdf

def two_element_fast_fading(num_samples_slow=1000, num_samples_fast=5000):
    fig, axs = plt.subplots()
    num_elements = 2
    dependencies = ["comon", "counter", "indep"]
    channel_realizations = rvs_channel_phases(num_elements, num_samples_slow)
    channel_realizations = np.tile(channel_realizations, (num_samples_fast, 1, 1))
    for _dependency in dependencies:
        print("Working on '{}'".format(_dependency))
        ris_phases = rvs_ris_phases(num_elements, num_samples_slow, num_samples_fast, copula=_dependency)
        total_phase = channel_realizations + ris_phases
        const_phase = gains_constant_phase(total_phase)
        capac_const_phase = np.log2(1 + const_phase)
        expect_capac = np.mean(capac_const_phase, axis=0)
        _hist = np.histogram(expect_capac, bins=100)
        _r_ax = np.linspace(0, 3, 1000)
        cdf_hist = stats.rv_histogram(_hist).cdf(_r_ax)
        cdf_exact = cdf_ergodic_capac_two_elements(_r_ax, copula=_dependency)
        axs.plot(_r_ax, cdf_hist, label="ECDF -- {}".format(_dependency))
        axs.plot(_r_ax, cdf_exact, '--', label="Exact -- {}".format(_dependency))
    axs.legend()
    axs.set_title("Artificial Fast Fading with N={:d} RIS Elements".format(num_elements))
    axs.set_xlabel("Rate $R$")
    axs.set_ylabel("Outage Probability")

The following plot shows the outage probability for the two-element RIS when phase hopping is performed.
Three different dependencies between the two phases $\theta_1$ and $\theta_2$ are considered.

1. _Comonotonicity:_ The two phases are set to the same value. This effectively reduces the two-element RIS to a one-element one.
2. _Independence:_ The two phases are set independently from one another.
3. _Counter-Monotonicity:_ The two phases are set to "opposite" values, i.e., $\theta_2=2\pi-\theta_1$.

In [10]:
two_element_fast_fading()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Working on 'comon'


<ipython-input-9-dd673a9dad76>:4: RuntimeWarning: invalid value encountered in arcsin
  cdf = 2/np.pi * np.arcsin(0.5 * np.sqrt(2**rate-1))


Working on 'counter'
Working on 'indep'


## N-Element RIS

After restricting the simulations to a two-element RIS, we now extend it to the general $N$-element case.

In [11]:
def n_element_fast_fading(num_samples_slow=1000, num_samples_fast=5000):
    fig, axs = plt.subplots()
    def update(num_elements=3):
        axs.clear()
        channel_realizations = rvs_channel_phases(num_elements, num_samples_slow)
        channel_realizations = np.tile(channel_realizations, (num_samples_fast, 1, 1))
        ris_phases = rvs_ris_phases(num_elements, num_samples_slow, num_samples_fast, copula="indep")
        channel_realizations = channel_realizations + ris_phases
        const_phase = gains_constant_phase(channel_realizations)
        capac_const_phase = np.log2(1 + const_phase)
        expect_capac = np.mean(capac_const_phase, axis=0)
        _hist = np.histogram(expect_capac, bins=100)
        _r_ax = np.linspace(0, 3, 1000)
        cdf_hist = stats.rv_histogram(_hist).cdf(_r_ax)
        axs.plot(_r_ax, cdf_hist)
        axs.set_title("Artificial Fast Fading with N={:d} RIS Elements".format(num_elements))
        axs.set_xlabel("Rate $R$")
        axs.set_ylabel("Outage Probability")
    interact_manual(update, num_elements=(2, 10, 1))

The following plot shows the outage probability for phase hopping of an $N$-element RIS with independently varied phases.

_Please note:_ In order to run the simulation, you need to first set the desired number of elements with the slider and then hit `Run Interact` to start the calculations.

In [13]:
n_element_fast_fading()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=3, description='num_elements', max=10, min=2), Button(description='Run I…

## Quantized Phases

In the previous examples, we only considered continuous phases with a uniform distribution over $[0, 2\pi]$.  
We now switch to the assumption that only a discrete set of RIS phases is available. They are given by uniformly quantizing $[0, 2\pi]$ in $K$ steps.
The individual RIS phases are then randomly drawn from this set, i.e.,
$$\theta_i \in Q=\left\{k \frac{2\pi}{K} \;\middle|\; k=0, \dots{}, K-1\right\}, \quad i=1, \dots{}, N.$$

In [16]:
def quantized_two_phases_two_elements(num_samples_slow=1000, num_samples_fast=50000):
    fig, axs = plt.subplots()
    num_elements = 2
    dependency = "indep"
    channel_realizations = rvs_channel_phases(num_elements, num_samples_slow)
    channel_realizations = np.tile(channel_realizations, (num_samples_fast, 1, 1))
    ris_phases = rvs_ris_phases_quant(num_elements, num_samples_slow, num_samples_fast,
                                      copula=dependency, K=2)
    total_phases = channel_realizations + ris_phases
    const_phase = gains_constant_phase(total_phases)
    capac_const_phase = np.log2(1 + const_phase)
    expect_capac = np.mean(capac_const_phase, axis=0)
    _hist = np.histogram(expect_capac, bins=100)
    _r_ax = np.linspace(0, 3, 1000)
    cdf_hist = stats.rv_histogram(_hist).cdf(_r_ax)
    zoc = 0.5*np.log2(5)
    axs.plot(_r_ax, cdf_hist, label="ECDF")
    axs.vlines(zoc, 0, 1, 'r', label="ZOC -- Exact")
    axs.legend()
    axs.set_title("Artificial Fast Fading with N={:d} RIS Elements\nQuantized Phases with 2 Quantization Steps".format(num_elements))
    axs.set_xlabel("Rate $R$")
    axs.set_ylabel("Outage Probability $\\varepsilon$")

The following plot shows the outage probability for a two-element RIS ($N=2$) with only two possible phase values ($K=2$).
The phases are i.i.d. uniformly varied.
For comparison, we additionally show the theoretical zero-outage capacity (ZOC).

_Please note:_ Since we only average over a finite number of fast-fading samples, there might be realizations that are less than the theoretical ZOC.

In [18]:
quantized_two_phases_two_elements(num_samples_fast=10000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …